## 【ポアソン回帰】冠動脈心疾患と喫煙習慣

冠動脈心疾患のデータを使って、ポアソン回帰を行ってみる。

In [ ]:
import pymc as pm
import arviz as az

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rcParams['font.size'] = 12
plt.rcParams['figure.figsize'] = [8, 6]

## Load  Data

In [ ]:
data = pd.read_csv('smoke.csv')

In [ ]:
data

## Preprocess & Scale Data

In [ ]:
agecat = data['agecat'].values
deaths = data['deaths'].values
smoke = data['smoke'].values

population = data['population'].values

In [ ]:
agecat_mu = np.mean(agecat)
agecat_sd = np.std(agecat)

agecat_scaled = (agecat - agecat_mu) / agecat_sd

## Check Data

In [ ]:
sns.scatterplot(x=agecat, y=deaths/population, hue=smoke, s=150)
plt.xticks(agecat)

plt.xlabel('Age Category')
plt.ylabel('Death Rate');

In [ ]:
sns.scatterplot(x=agecat, y=np.log(deaths/population), hue=smoke, s=150)
plt.xticks(agecat)

plt.xlabel('Age Category')
plt.ylabel('log( Death Rate )');

## Poisson Regression

In [ ]:
with pm.Model() as model:

    b1 = pm.Normal('b1', mu=0, sigma=10)
    b2 = pm.Normal('b2', mu=0, sigma=10)
    b3 = pm.Normal('b3', mu=0, sigma=10)

    theta = b1 + b2 * smoke + b3 * agecat_scaled

    mu = pm.math.exp(theta) * population

    obs = pm.Poisson('obs', mu=mu, observed=deaths)

In [ ]:
with model:

    trace = pm.sample(3000, tune=6000, return_inferencedata=False)
    idata = pm.to_inference_data(trace, log_likelihood=True)

## Check MCMC-samples

In [ ]:
az.plot_trace(idata)
plt.gcf().subplots_adjust(wspace=0.5, hspace=0.5)

In [ ]:
az.summary(idata)

In [ ]:
az.plot_posterior(idata, var_names=['b2', 'b3'], ref_val=0);

## Posterior Predictive Check

In [ ]:
with model:

    ppc = pm.sample_posterior_predictive(idata, return_inferencedata=False)

In [ ]:
ppc['obs'].shape

In [ ]:
fig = plt.figure(figsize=(12, 12))

for k in range(10):

    ax = fig.add_subplot(5, 2, k+1)

    az.plot_dist(ppc['obs'][:, :, k])
    ax.axvline(deaths[k], color='r', linestyle='dashed')
    ax.set_title('Age Category = {}, Smoke = {}'.format(agecat[k], smoke[k]))

plt.tight_layout()

## Export Inference Data

In [ ]:
idata.to_netcdf('idata_base.nc')